In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from benchmarkrewriter.benchmark_parser import BenchmarkParser, WorkerBenchmarkParser, WorkerEncoding
import os

path_to_results = r'C:\Users\huda\Downloads\\'
path_to_results = r'C:\Users\huda\Downloads\ga\ga\\'
benchmark_path = r'C:\Users\huda\Documents\GitHub\scheduling_model_jrc\code\upgrades\benchmarks_with_workers'

In [2]:
import math
def multiset_permutations(sequence : list) -> int:
    a = math.factorial(len(sequence))
    counts = [sequence.count(i) for i in range(min(sequence), max(sequence)+1)]
    b = math.prod([math.factorial(counts[x]) for x in range(len(counts))])
    return a/b

print(multiset_permutations([0, 0, 1, 1]))
print(multiset_permutations([1, 1, 2, 2]))
print(multiset_permutations([1, 1]))
print(multiset_permutations([1, 2]))
print(multiset_permutations([1]))
print(multiset_permutations([1, 1, 2]))
print(multiset_permutations([1, 2, 1]))
print(multiset_permutations([1, 2, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7 ,8, 8, 8]))

6.0
6.0
1.0
2.0
1.0
3.0
3.0
1.30729041081037e+29


In [3]:
def worker_flexibility(benchmark : WorkerEncoding):
    n_assignments = 0
    m = benchmark.n_machines()
    o = benchmark.n_operations()
    durations = benchmark.durations()
    w = durations.shape[2]
    for i in range(len(durations)):
        for j in range(len(durations[i])):
            for k in range(len(durations[i][j])):
                if durations[i][j][k] > 0:
                    n_assignments += 1
    average_assignments = n_assignments / o
    return average_assignments / (m*w)


def get_flexibility_and_dv_worker(benchmark):
    all = 0
    unique = []
    machines_available = 0
    durations = benchmark.durations()
    for i in range(len(durations)):
        for j in range(len(durations[i])):
            for k in range(len(durations[i][j])):
                if durations[i][j][k] > 0:
                    if durations[i][j][k] not in unique:
                        unique.append(durations[i][j][k])
                    all += 1
            if any([x > 0 for x in durations[i][j]]):
                machines_available+=1
    return worker_flexibility(benchmark), len(unique) / all

def remap(name):
    if name.startswith('_'):
        name = name[1:]
    values = name.split('_')
    if values[0].startswith('Behnke'):
        return 'Behnke'+values[1]
    if values[0].startswith('Brandimarte'):
        return 'BrandimarteMk'+values[1]
    if values[0].startswith('Chambers'):
        return 'ChambersBarnes'+values[1]
    if values[0].startswith('HurinkS'):
        return 'HurinkSdata'+values[1]
    if values[0].startswith('HurinkE'):
        return 'HurinkEdata'+values[1]
    if values[0].startswith('HurinkR'):
        return 'HurinkRdata'+values[1]
    if values[0].startswith('HurinkV'):
        return 'HurinkVdata'+values[1]
    if values[0].startswith('DP'):
        return 'DPpaulli'+values[1]
    if values[0].startswith('Kacem'):
        return 'Kacem'+values[1]
    if values[0].startswith('Fattahi'):
        return 'Fattahi'+values[1]
    return name

def read_benchmarks_workers(path):
    result = dict()
    benchmarks = os.listdir(path)
    for benchmark in benchmarks:
        #parser = BenchmarkParser()
        parser = WorkerBenchmarkParser()
        data = parser.parse_benchmark(path + '/' + benchmark)
        #f, dv = get_flexibility_and_dv(data)
        f, dv = get_flexibility_and_dv_worker(data)
        instance_name = remap(benchmark[2:-12])
        result[instance_name] = {'n_operations': data.n_operations(), 'flexibility': f, 'duration_variety': dv, 'n_machines': data.n_machines()}

    return result

In [4]:
benchmark_data = read_benchmarks_workers(benchmark_path)

In [5]:
import os
def rewrite(category):
    files = os.listdir(path_to_results+category)
    json_data = {}
    for file in files:
        with open(path_to_results + category + "\\" + file, 'r') as f:
            content = f.read()
            content = '{"' + content[:7] + '":' + content[8:-1] + ']' + '}'
            json_data[file] = content
    return json_data

In [6]:
def remap(name):
    if name.startswith('_'):
        name = name[1:]
    values = name.split('_')
    if values[1].startswith('Behnke'):
        return 'Behnke'+values[-1]
    if values[1].startswith('Brandimarte'):
        return 'BrandimarteMk'+values[-1]
    if values[1].startswith('Chambers'):
        return 'ChambersBarnes'+values[-1]
    if values[1].startswith('Hurink') and values[2].startswith('s'):
        return 'HurinkSdata'+values[-1]
    if values[1].startswith('HurinkE') and values[2].startswith('e'):
        return 'HurinkEdata'+values[-1]
    if values[1].startswith('HurinkR') and values[2].startswith('e'):
        return 'HurinkRdata'+values[-1]
    if values[1].startswith('HurinkV') and values[2].startswith('v'):
        return 'HurinkVdata'+values[-1]
    if values[1].startswith('DP'):
        return 'DPpaulli'+values[-1]
    if values[1].startswith('Kacem'):
        return 'Kacem'+values[-1]
    if values[1].startswith('Fattahi'):
        return 'Fattahi'+values[-1]
    return name

In [7]:
import json
results_categories = os.listdir(path_to_results)
results_categories = ['local', 'nolocal']
results = dict()
for category in results_categories:
    results[category] = dict()
    instances = os.listdir(path_to_results + category)
    for instance in instances:
        #with open(path_to_results + category + '\\' + instance, 'r') as f:
        #TODO: turn into valid json
        j_data = rewrite(category)
        for instance in j_data:
            ga_data = json.loads(j_data[instance])
            # TODO: gather best, average, ...
            if instance not in results[category]:
                results[category][remap(instance[:-17])] = ga_data


In [8]:
for result in results:
    print(results[result].keys())

dict_keys(['Behnke42', 'Behnke46', 'Behnke60', 'BrandimarteMk12', 'BrandimarteMk14', 'BrandimarteMk7', 'HurinkSdata18', 'HurinkSdata1', 'HurinkSdata38', 'HurinkSdata40', 'HurinkSdata54', 'HurinkSdata61', 'HurinkSdata63', '2b_Hurink_edata_1', '2b_Hurink_edata_6', '2c_Hurink_rdata_28', '2c_Hurink_rdata_38', '2c_Hurink_rdata_50', '2d_Hurink_vdata_18', '2d_Hurink_vdata_5', 'DPpaulli15', 'DPpaulli18', 'DPpaulli1', 'DPpaulli9', 'ChambersBarnes10', 'Kacem3', 'Kacem4', 'Fattahi14', 'Fattahi20'])
dict_keys(['Behnke42', 'Behnke46', 'Behnke60', 'BrandimarteMk12', 'BrandimarteMk14', 'BrandimarteMk7', 'HurinkSdata18', 'HurinkSdata1', 'HurinkSdata38', 'HurinkSdata40', 'HurinkSdata54', 'HurinkSdata61', 'HurinkSdata63', '2b_Hurink_edata_1', '2b_Hurink_edata_6', '2c_Hurink_rdata_28', '2c_Hurink_rdata_38', '2c_Hurink_rdata_50', '2d_Hurink_vdata_18', '2d_Hurink_vdata_5', 'DPpaulli15', 'DPpaulli18', 'DPpaulli1', 'DPpaulli9', 'ChambersBarnes10', 'Kacem3', 'Kacem4', 'Fattahi14', 'Fattahi20'])


In [15]:
# plot best and average results

# plot all best results and best average results for every instance, colored in the category that performed best
labels = list(results.keys())
best_category = dict()
#for instance in benchmark_data:
#    best_category[category] = []
for category in results:
    best_category[category] = []
for instance in benchmark_data:
    best_overall = float('inf')
    best_categories = []
    for category in results:
        
        if instance in results[category]:
            # TODO: decide what to do with instances with multiple best, also do same for average
            best = float('inf')
            average = 0
            for result in results[category][instance]['results']:
                if result['Result']['Fitness']['Makespan'] < best:
                    best = result['Result']['Fitness']['Makespan']
                average += result['Result']['Fitness']['Makespan']
            average /= len(results[category][instance]['results'])
            if best < best_overall:
                best_overall = best
                best_categories = []
                best_categories.append(category)
            elif best_overall == best:
                best_categories.append(category)
            for category in best_categories:
                best_category[category].append(instance)

for category in best_category:
    n = [x['n_operations'] for x in best_category[category]]
    m = [x['n_machines'] for x in best_category[category]]
    plt.scatter(n, m, label=category)
    plt.xlabel('# Operations')
    plt.ylabel('# Machines')
    #...
plt.show()

TypeError: string indices must be integers

In [39]:
best_overall = dict()
average_overall = dict()
tied_overall = []
average_tied_overall = []
for instance in benchmark_data:
    if instance in results['local'] or instance in results['nolocal']:
        best_overall[instance] = ('None', float('inf'))
        average_overall[instance] = ('None', float('inf'))
    for category in results:
        
        if instance in results[category]:
            # TODO: decide what to do with instances with multiple best, also do same for average
            best = float('inf')
            average = 0
            for result in results[category][instance]['results']:
                if result['Result']['Fitness']['Makespan'] < best:
                    best = result['Result']['Fitness']['Makespan']
                average += result['Result']['Fitness']['Makespan']
            average /= len(results[category][instance]['results'])
            if best < best_overall[instance][1]:
                best_overall[instance] = (category, best)
            elif best == best_overall[instance][1]:
                tied_overall.append(instance)
                best_overall[instance] = ('None', float('inf'))
            if average < average_overall[instance][1]:
                average_overall[instance] = (category, average)
            elif average == average_overall[instance][1]:
                average_tied_overall.append(instance)
                average_overall[instance] = ('None', float('inf'))

In [40]:
local_win = [best_overall[x] for x in best_overall if best_overall[x][0] == 'local']
nolocal_win = [best_overall[x] for x in best_overall if best_overall[x][0] == 'nolocal']
local_average_win = [average_overall[x] for x in average_overall if average_overall[x][0] == 'local']
nolocal_average_win = [average_overall[x] for x in average_overall if average_overall[x][0] == 'nolocal']

In [41]:
for result in best_overall:
    if result[0] == 'nolocal':
        print('found')

In [42]:
print(f'Local better: {len(local_win)}')
print(f'No Local better: {len(nolocal_win)}')
print(f'Average Local better: {len(local_average_win)}')
print(f'Average No Local better: {len(nolocal_average_win)}')

Local better: 4
No Local better: 10
Average Local better: 1
Average No Local better: 13


In [44]:
len(tied_overall)

1

In [45]:
len(average_tied_overall)

1

In [46]:
tied_overall

['Kacem3']

In [47]:
average_tied_overall

['Kacem3']